<center>

## Erro de português
>Quando o português chegou \\
>debaixo duma bruta chuva \\
>vestiu o índio \\
>que pena! \\
>fosse uma manhã de sol \\
>o índio tinha despido o português \\

</center>

<right>Oswald de Andrade, 1927</right>

\\

---

\\

Aquaplanagem, menor visibilidade, risco de derrapamento: condições climáticas advsersas podem impactar a maneira como pessoas dirigem e a segurança na via. Além do senso comum, o manual de direção defensiva recomenda uma maior distância de seguimento e menor velocidade em condições de tempo adversas. No célebre poema "Erro de Português" apresentado acima, Oswald de Andrade (1890-1954) entreteve: _"fosse uma manhã de sol..."_

\\

Com este estudo, pretendo uma resposta tentativa a esse questionamento: **Em um mundo contrafactual, onde os motoristas não estão sujeitos às intempéries, observaríamos a mesma taxa de fatalidade em acidentes nas estradas brasileiras?**

\\

Para responder a essa questão eu vou utilizar o *paradigma de desfechos potenciais* (potential outcomes), também conhecido por *modelo causal de Neyman-Rubin*, aplicados a dados de acidentes de trânsito em território nacional, coletados entre 2007–2021 e compilados pela Polícia Rodoviária Federal (PRF), gentilmente [disponibilizados como um data set público no Kaggle](https://www.kaggle.com/datasets/mcamera/brazil-highway-traffic-accidents/data).


\\

---

\\


# Interações esperadas

Nossas interação causa-efeito alvo será entre as seguintes variáveis: condição climática e um "score de fatalidade" (quociente entre o número de vítimas fatais em uma ocorrência, coluna _mortos_, e o número de passageiros envolvidos, coluna _pessoas_).

Também iremos nos atentar a outros fatores que podem influenciar um desfecho mais sério:
- o momento do dia (coluna *fase_dia*) pode afetar a atenção (via modulação da atenção dos motoristas por conta do ritmo circadiano), luminosidade e movimento na via;
- o tipo de pista (coluna *tipo_pista*), que pode determinar o tipo de manobra e o envolvimento de outros veículos; [SUPRIMIDO POR HORA]
- a morfologia da pista (coluna *tracado_via*), uma vez que a presença de curvas e cruzamentos podem exacerbar as dificuldades em climas adversos;
- a classificação da via (urbana ou rural; coluna *uso_solo*) pode influenciar a velocidade de resgate e atendimento, influenciando o desfecho de saúde dos envolvidos.

\\

---

\\

**Algumas inspiraçoes teóricas:**

Igelström E, Craig P, Lewsey J, et al. _"Causal inference and effect estimation using observational data"_. J Epidemiol Community Health 2022;76:960–966.

"Causal inference with observational data: A tutorial on propensity score analysis", doi.org/10.1016/j.leaqua.2023.101678

# Configurações e preparação

In [ ]:
# INSTALLS

In [ ]:
#pip install unidecode

In [ ]:
# IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import json
import scipy
import seaborn as sns

# ACCESS TO DATA
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Agregar os dados de vários anos em um único DataFrame

In [ ]:
anos = list(range(2007,2022,1))

In [ ]:
acidentes = [] # todos acidentes registrados entre 2007-2021
for ano in anos:
  acidentes_ano = pd.read_csv(f'./drive/MyDrive/data/acidentes/datatran{ano}.csv', sep = ';', encoding = 'latin1', decimal = ',').copy()
  acidentes_ano['ano'] = ano

  acidentes.append(acidentes_ano)
  del acidentes_ano # limpar a memória

<ipython-input-3-50d0a9a5aa47>:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  acidentes_ano = pd.read_csv(f'./drive/MyDrive/data/acidentes/datatran{ano}.csv', sep = ';', encoding = 'latin1', decimal = ',').copy()
<ipython-input-3-50d0a9a5aa47>:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  acidentes_ano = pd.read_csv(f'./drive/MyDrive/data/acidentes/datatran{ano}.csv', sep = ';', encoding = 'latin1', decimal = ',').copy()
<ipython-input-3-50d0a9a5aa47>:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  acidentes_ano = pd.read_csv(f'./drive/MyDrive/data/acidentes/datatran{ano}.csv', sep = ';', encoding = 'latin1', decimal = ',').copy()


In [ ]:
acidentes_global = pd.concat(acidentes).copy()
print(f'Há {len(acidentes_global)} acidentes registrados no período 2007-2021')

Há 1882953 acidentes registrados no período 2007-2021


In [ ]:
acidentes_global.columns

Index(['id', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km',
       'municipio', 'causa_acidente', 'tipo_acidente',
       'classificacao_acidente', 'fase_dia', 'sentido_via',
       'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo',
       'ano', 'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos',
       'ignorados', 'feridos', 'veiculos', 'latitude', 'longitude', 'regional',
       'delegacia', 'uop'],
      dtype='object')

# Modificações previstas no conjunto de dados

1. Criar uma coluna "score de fatalidade" como delineado acima (vítimas fatais/número de envolvidos no acidente) ✔
2. Omitir as demais colunas (i.e. _drop_ em uma cópia do dataframe) ✔
3. Remover linhas com valor nulo para as variáveis de interesse (no futuro pode-se fazer uma inputação; agora não disponho de tanto tempo); ✔
4. Transformar os rótulos textuais da condição climática para um score: 1 = {céu claro, sol}, 2 = {nublado, vento, garoa/chuvisco}, 3 = {chuva, neve, nevoeiro/neblina, granizo}; ✔
5. Fazer transformações similares de rótulo textual para integer. Convenção: _valores menores serão imputados a condições menos adversas (e.g. via reta, pleno dia, via urbana, etc.) e valores mais altos a condições mais adversas (e.g. pista curva, plena noite, via rural, etc.).

In [ ]:
# 1. Criar uma coluna "score de fatalidade" como delineado acima (vítimas fatais/número de envolvidos no acidente) ✔
acidentes_global['score_fatalidade'] = acidentes_global['mortos']/acidentes_global['pessoas']

In [ ]:
# 2. Omitir as demais colunas

target_vars = ['fase_dia','condicao_metereologica','tipo_pista','tracado_via','uso_solo','score_fatalidade']

acidentes_df = acidentes_global[target_vars].copy()

In [ ]:
# 3. Remover linhas com valor nulo para as variáveis de interesse (no futuro pode-se fazer uma inputação; agora não disponho de tanto tempo)

# O dataframe tem variação nesses rótulos por conta de unicode/capitalização. Vamos modificar isso tirando os acentos e capitalizando tudo
for col in ['condicao_metereologica','fase_dia','tipo_pista','tracado_via','uso_solo']:
  acidentes_df[col] = acidentes_df[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
  acidentes_df[col] = acidentes_df[col].str.upper()

#print(acidentes_df.condicao_metereologica.unique()) # A var. causal tem valores NaN e strings '(null)' e 'Ignorada' que não são informativos e vamos remover as linhas que os contenham
print('# antes da filtragem:',len(acidentes_df))

# antes da filtragem: 1882953


In [ ]:
# Remover linhas cujo valor na coluna-alvo esteja na lista de valores passada
def filter_entry_by_column_value(df, column, values):
  lines_with_condition = df[column].isin(values)
  return df[~lines_with_condition].copy()

# Variável de interesse (causal)
acidentes_df = filter_entry_by_column_value(acidentes_df,'condicao_metereologica',values=['(NULL)','IGNORADA','IGNORADO',np.nan])

# Outras variáveis
acidentes_df = filter_entry_by_column_value(acidentes_df,'tracado_via',values=['(NULL)','NAO INFORMADO',np.nan])
acidentes_df = filter_entry_by_column_value(acidentes_df,'tipo_pista',values=['(NULL)'])
acidentes_df = filter_entry_by_column_value(acidentes_df,'fase_dia',values=['(NULL)',np.nan])
acidentes_df = filter_entry_by_column_value(acidentes_df,'uso_solo',values=['NAO','SIM']) # não sabemos o que são as tags "não"/"sim"

#print(acidentes_df.condicao_metereologica.unique())
print('# depois da filtragem:',len(acidentes_df))

# depois da filtragem: 1533655


In [ ]:
# 4. Transformar os rótulos textuais da condição climática para um score: 1 = {céu claro, sol}, 2 = {nublado, vento, garoa/chuvisco}, 3 = {chuva, neve, nevoeiro/neblina, granizo}

# Definir os novos rótulos

# Condições metereológicas:
condicoes_dict = {'CEU CLARO':1,'SOL':1,
                  'NUBLADO':2,'VENTO':2,'GAROA/CHUVISCO':2,
                  'CHUVA':3,'NEVOEIRO/NEBLINA':3,'GRANIZO':3,'NEVE':3}

# Uso do solo:
uso_solo_dict = {'RURAL': 0, 'URBANO':1}

# Fase do dia:
fase_dia_dict = {'PLENO DIA': 1,
                 'AMANHECER':2,'ANOITECER':2,
                 'PLENA NOITE':3}

# Tipo de pista:
tipo_pista_dict = {'SIMPLES': 1,'DUPLA': 1,'MULTIPLA': 1} # não me convenci imediatamente de que uma oferece mais periculosidade que a outra então por enquanto deixo todas com mesmo valor

# Traçado da via:
# Aplicar apenas à reta um valor menor (decisão arbitrária)
tracados_dict = {tracado:2 for tracado in acidentes_df.tracado_via.unique()}
tracados_dict.update({'RETA':1})

# Fazer as substituições
acidentes_df.replace(to_replace={'condicao_metereologica':condicoes_dict},inplace=True)
acidentes_df.replace(to_replace={'uso_solo':uso_solo_dict},inplace=True)
acidentes_df.replace(to_replace={'tracado_via':tracados_dict},inplace=True)
acidentes_df.replace(to_replace={'fase_dia':fase_dia_dict},inplace=True)

In [ ]:
# Aparência do df
acidentes_df.head(10)

,fase_dia,condicao_metereologica,tipo_pista,tracado_via,uso_solo,score_fatalidade
0,1,1,DUPLA,1,0,0.0
1,1,3,DUPLA,1,0,0.0
2,1,1,SIMPLES,1,0,0.0
3,3,1,SIMPLES,1,1,0.4
4,2,1,DUPLA,1,0,0.0
5,2,3,SIMPLES,2,0,0.0
6,1,1,SIMPLES,2,1,0.0
7,3,1,MULTIPLA,1,1,0.0
8,3,3,SIMPLES,2,0,0.0
9,3,2,SIMPLES,1,1,0.0
